In [86]:
import time
import unittest
from test.test_meteorology import Test_relhum

import cupy as cp
import dask
import numpy as np
import pandas as pd
import src.geocat.comp.meteorology as geo
import xarray as xr
from numba import jit,njit,cuda

In [2]:
# import dask.array as da
from dask_jobqueue import PBSCluster
from dask.distributed import Client

clusterCPU = PBSCluster(memory='50 GB',
                     processes=1,
                     cores=1,
                     queue='casper',
                     walltime='02:00:00',
                     resource_spec='select=1:ncpus=1:ngpus=1:mem=50gb')
print(clusterCPU.job_script())
clusterCPU.scale(1)
client = Client(clusterCPU)
#cluster.close()
client

/glade/work/hkashgar/conda-envs/geocat/lib/python3.10/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q casper
#PBS -A NTDD0005
#PBS -l select=1:ncpus=1:ngpus=1:mem=50gb
#PBS -l walltime=02:00:00

/glade/work/hkashgar/conda-envs/geocat/bin/python -m distributed.cli.dask_worker tcp://10.12.205.44:33210 --nthreads 1 --memory-limit 46.57GiB --name dummy-name --nanny --death-timeout 60 --interface ib0 --protocol tcp://



Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hkashgar/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hkashgar/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.205.44:33210,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hkashgar/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# _Relhum

In [87]:
i = 5
p = np.random.uniform(low=2000,high=100800,size=10**i)
t = np.random.uniform(low=194.65,high=302.45,size=10**i)
w = np.random.uniform(low=0,high=0.02038,size=10**i)
xp = np

def _relhum(t,w,p):
    table = np.array([
        0.01403, 0.01719, 0.02101, 0.02561, 0.03117, 0.03784, 0.04584, 0.05542,
        0.06685, 0.08049, 0.09672, 0.1160, 0.1388, 0.1658, 0.1977, 0.2353,
        0.2796, 0.3316, 0.3925, 0.4638, 0.5472, 0.6444, 0.7577, 0.8894, 1.042,
        1.220, 1.425, 1.662, 1.936, 2.252, 2.615, 3.032, 3.511, 4.060, 4.688,
        5.406, 6.225, 7.159, 8.223, 9.432, 10.80, 12.36, 14.13, 16.12, 18.38,
        20.92, 23.80, 27.03, 30.67, 34.76, 39.35, 44.49, 50.26, 56.71, 63.93,
        71.98, 80.97, 90.98, 102.1, 114.5, 128.3, 143.6, 160.6, 179.4, 200.2,
        223.3, 248.8, 276.9, 307.9, 342.1, 379.8, 421.3, 466.9, 517.0, 572.0,
        632.3, 698.5, 770.9, 850.2, 937.0, 1032.0, 1146.6, 1272.0, 1408.1,
        1556.7, 1716.9, 1890.3, 2077.6, 2279.6, 2496.7, 2729.8, 2980.0, 3247.8,
        3534.1, 3839.8, 4164.8, 4510.5, 4876.9, 5265.1, 5675.2, 6107.8, 6566.2,
        7054.7, 7575.3, 8129.4, 8719.2, 9346.50, 10013.0, 10722.0, 11474.0,
        12272.0, 13119.0, 14017.0, 14969.0, 15977.0, 17044.0, 18173.0, 19367.0,
        20630.0, 21964.0, 23373.0, 24861.0, 26430.0, 28086.0, 29831.0, 31671.0,
        33608.0, 35649.0, 37796.0, 40055.0, 42430.0, 44927.0, 47551.0, 50307.0,
        53200.0, 56236.0, 59422.0, 62762.0, 66264.0, 69934.0, 73777.0, 77802.0,
        82015.0, 86423.0, 91034.0, 95855.0, 100890.0, 106160.0, 111660.0,
        117400.0, 123400.0, 129650.0, 136170.0, 142980.0, 150070.0, 157460.0,
        165160.0, 173180.0, 181530.0, 190220.0, 199260.0, 208670.0, 218450.0,
        228610.0, 239180.0, 250160.0, 261560.0, 273400.0, 285700.0, 298450.0,
        311690.0, 325420.0, 339650.0, 354410.0, 369710.0, 385560.0, 401980.0,
        418980.0, 436590.0, 454810.0, 473670.0, 493170.0, 513350.0, 534220.0,
        555800.0, 578090.0, 601130.0, 624940.0, 649530.0, 674920.0, 701130.0,
        728190.0, 756110.0, 784920.0, 814630.0, 845280.0, 876880.0, 909450.0,
        943020.0, 977610.0, 1013250.0, 1049940.0, 1087740.0, 1087740.
    ])

    maxtemp = 375.16
    mintemp = 173.16

    t = xp.clip(t, mintemp, maxtemp)

    it = (t - mintemp).astype(np.int64)
    t2 = mintemp + it

    es = (t2 + 1 - t) * table[it] + (t - t2) * table[it + 1]
    es = es * 0.1

    rh = (w * (p - 0.378 * es) / (0.622 * es)) * 100

    rh = np.clip(rh, 0.0001, None)
    return rh

In [20]:
np.allclose(_relhum(t,w,p),geo.relhum(t,w,p))

True

# Numba, CPU

In [28]:
from numba import njit,jit,vectorize
f_jit = jit(nopython=True,parallel=True)(_relhum)

In [33]:
%time _relhum(t,w,p)

CPU times: user 3.27 ms, sys: 0 ns, total: 3.27 ms
Wall time: 2.74 ms


array([1.57902441e+02, 1.01612509e+06, 6.63077059e+00, ...,
       4.96252469e+03, 4.92476087e+01, 6.42990073e+02])

In [31]:
%time f_jit(t, w, p)

CPU times: user 3.58 ms, sys: 0 ns, total: 3.58 ms
Wall time: 1.34 ms


array([1.57902441e+02, 1.01612509e+06, 6.63077059e+00, ...,
       4.96252469e+03, 4.92476087e+01, 6.42990073e+02])

In [ ]:
from numba import njit,jit,vectorize
f_jit = njit()(_relhum)

In [ ]:
%time f_jit(t, w, p)

# Numba, kernel

In [88]:
from numba import cuda
@cuda.jit()
def _relhum_gpu(t,w,p,rh):
    pos = cuda.grid(1)
    table = np.array([
        0.01403, 0.01719, 0.02101, 0.02561, 0.03117, 0.03784, 0.04584, 0.05542,
        0.06685, 0.08049, 0.09672, 0.1160, 0.1388, 0.1658, 0.1977, 0.2353,
        0.2796, 0.3316, 0.3925, 0.4638, 0.5472, 0.6444, 0.7577, 0.8894, 1.042,
        1.220, 1.425, 1.662, 1.936, 2.252, 2.615, 3.032, 3.511, 4.060, 4.688,
        5.406, 6.225, 7.159, 8.223, 9.432, 10.80, 12.36, 14.13, 16.12, 18.38,
        20.92, 23.80, 27.03, 30.67, 34.76, 39.35, 44.49, 50.26, 56.71, 63.93,
        71.98, 80.97, 90.98, 102.1, 114.5, 128.3, 143.6, 160.6, 179.4, 200.2,
        223.3, 248.8, 276.9, 307.9, 342.1, 379.8, 421.3, 466.9, 517.0, 572.0,
        632.3, 698.5, 770.9, 850.2, 937.0, 1032.0, 1146.6, 1272.0, 1408.1,
        1556.7, 1716.9, 1890.3, 2077.6, 2279.6, 2496.7, 2729.8, 2980.0, 3247.8,
        3534.1, 3839.8, 4164.8, 4510.5, 4876.9, 5265.1, 5675.2, 6107.8, 6566.2,
        7054.7, 7575.3, 8129.4, 8719.2, 9346.50, 10013.0, 10722.0, 11474.0,
        12272.0, 13119.0, 14017.0, 14969.0, 15977.0, 17044.0, 18173.0, 19367.0,
        20630.0, 21964.0, 23373.0, 24861.0, 26430.0, 28086.0, 29831.0, 31671.0,
        33608.0, 35649.0, 37796.0, 40055.0, 42430.0, 44927.0, 47551.0, 50307.0,
        53200.0, 56236.0, 59422.0, 62762.0, 66264.0, 69934.0, 73777.0, 77802.0,
        82015.0, 86423.0, 91034.0, 95855.0, 100890.0, 106160.0, 111660.0,
        117400.0, 123400.0, 129650.0, 136170.0, 142980.0, 150070.0, 157460.0,
        165160.0, 173180.0, 181530.0, 190220.0, 199260.0, 208670.0, 218450.0,
        228610.0, 239180.0, 250160.0, 261560.0, 273400.0, 285700.0, 298450.0,
        311690.0, 325420.0, 339650.0, 354410.0, 369710.0, 385560.0, 401980.0,
        418980.0, 436590.0, 454810.0, 473670.0, 493170.0, 513350.0, 534220.0,
        555800.0, 578090.0, 601130.0, 624940.0, 649530.0, 674920.0, 701130.0,
        728190.0, 756110.0, 784920.0, 814630.0, 845280.0, 876880.0, 909450.0,
        943020.0, 977610.0, 1013250.0, 1049940.0, 1087740.0, 1087740.
    ])

    maxtemp = 375.16
    mintemp = 173.16

    if(pos<t.size):
        t[pos] = xp.clip(t[pos], mintemp, maxtemp)

        it = (t[pos] - mintemp).astype(np.int64)
        t2 = mintemp + it

        es = (t2 + 1 - t[pos]) * table[it] + (t[pos] - t2) * table[it + 1]
        es = es * 0.1

        rh[pos] = (w[pos] * (p[pos] - 0.378 * es) / (0.622 * es)) * 100

        rh[pos] = np.clip(rh[pos], 0.0001, None)

In [89]:
rh_res_cpu = _relhum(t,w,p)

In [90]:
threadsperblock = 512
blockspergrid = (p.size + (threadsperblock - 1)) // threadsperblock
rh = np.zeros(p.shape)
_relhum_gpu[blockspergrid,threadsperblock](t, w, p,rh)

TypingError: Failed in cuda mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(list(float64)<iv=None>)
 
There are 4 candidate implementations:
      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
        With argument(s): '(list(float64)<iv=None>)':
       Rejected as the implementation raised a specific error:
         NumbaTypeError: Failed in nopython mode pipeline (step: nopython frontend)
       Only accept returning of array passed into the function as argument
  raised from /glade/work/hkashgar/conda-envs/geocat/lib/python3.10/site-packages/numba/core/typed_passes.py:150

During: resolving callee type: Function(<built-in function array>)
During: typing of call at /glade/scratch/hkashgar/ipykernel_173858/2775961184.py (5)


File "../../../../../scratch/hkashgar/ipykernel_173858/277596